# 15. 평가
- LLM(Large Language Model) 평가는 인공지능 언어 모델의 성능, 정확성, 일관성 및 기타 중요한 측면을 측정하고 분석하는 과정
- 이는 모델의 개선, 비교, 선택 및 응용 프로그램에 적합한 모델 결정에 필수적인 단계

<br>

#### 평가 방법
1. 자동화된 메트릭: BLEU, ROUGE, METEOR, SemScore 등의 지표를 사용
2. **인간 평가**: 전문가나 크라우드소싱을 통한 직접적인 평가를 수행
3. **작업 기반 평가**: 특정 작업에서의 성능을 측정
4. **LLM-as-judge**: 다른 LLM을 평가자로 사용하는 방법

<br>

#### LangChain에서의 Evaluation
1. **모듈화된 평가 컴포넌트**: 다양한 평가 방법을 쉽게 구현하고 조합
2. **Chain 평가**: 전체 LLM 애플리케이션 파이프라인을 평가
3. **데이터셋 기반 평가**: 사용자 정의 데이터셋을 사용하여 모델을 평가
4. **평가 지표**: 정확성, 일관성, 관련성 등 다양한 지표를 제공

<br>

### LLM-as-judge
- LLM-as-judge는 다른 LLM의 출력을 평가하기 위해 LLM을 사용

<br>

1. **자동화**: 인간의 개입 없이 대규모 평가를 수행
2. **일관성**: 평가 기준을 일관되게 적용
3. **유연성**: 다양한 평가 기준과 상황에 적응
4. **비용 효율성**: 인간 평가자에 비해 비용이 적게 들 수 있음

<br>

#### LLM-as-judge의 작동 방식
1. **입력 제공**: 평가할 LLM의 출력과 평가 기준을 제공
2. **분석**: 평가자 LLM이 제공된 출력을 분석
3. **평가**: 정의된 기준에 따라 점수나 피드백을 생성
4. **결과 집계**: 여러 평가 결과를 종합하여 최종 평가를 도출

<br>

#### 평가의 중요성
1. **모델 개선**: 약점을 식별하고 개선 방향을 제시
2. **신뢰성 확보**: 모델의 성능과 한계를 이해하는 데 도움
3. **적합한 모델 선택**: 특정 작업이나 도메인에 가장 적합한 모델을 선택
4. **윤리적 고려사항**: 편향, 공정성 등의 윤리적 측면을 평가

<br>

<hr>

<br>


## 15-01. 합성 테스트 데이터셋 생성 (RAGAS)
- RAG(검색 증강 생성) 증강 파이프라인의 성능을 평가하는 것은 매우 중요
- **그러나 문서에서 수백 개의 QA(질문-문맥-응답) 샘플을 수동으로 생성하는 것은 시간과 노동력이 많이 소요되며,**
  
  **또한 사람이 만든 질문은 철저한 평가에 필요한 복잡성 수준에 도달하기 어려워 궁극적으로 평가의 품질에 영향을 미칠 수 있음**

  $\rightarrow$ **합성 데이터 생성을 사용하면 데이터 집계 프로세스에서 개발자의 시간을 감소**

In [1]:
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

<br>

#### 문서 전처리

In [2]:
from langchain_community.document_loaders import PDFPlumberLoader

In [3]:
# pip install pdfplumber

In [4]:
loader = PDFPlumberLoader("data/SPRI_AI_Brief_2023년12월호_F.pdf")

docs = loader.load()
docs = docs[3:-1]

len(docs)

19

- 각 문서 객체에는 `metadata` 를 통해 액세스할 수 있는 문서에 대한 추가 정보를 저장하는 데 사용할 수 있는 메타데이터 사전이 포함되어 있으며,
  
  **메타데이터의 `filename` 속성이 필요**

In [5]:
for doc in docs[:5]:
    print(doc.metadata)

{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pdf', 'file_path': 'data/SPRI_AI_Brief_2023년12월호_F.pdf', 'page': 3, 'total_pages': 23, 'Author': 'dj', 'Creator': 'Hwp 2018 10.0.0.13462', 'Producer': 'Hancom PDF 1.3.0.542', 'CreationDate': "D:20231208132838+09'00'", 'ModDate': "D:20231208132838+09'00'", 'PDFVersion': '1.4'}
{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pdf', 'file_path': 'data/SPRI_AI_Brief_2023년12월호_F.pdf', 'page': 4, 'total_pages': 23, 'Author': 'dj', 'Creator': 'Hwp 2018 10.0.0.13462', 'Producer': 'Hancom PDF 1.3.0.542', 'CreationDate': "D:20231208132838+09'00'", 'ModDate': "D:20231208132838+09'00'", 'PDFVersion': '1.4'}
{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pdf', 'file_path': 'data/SPRI_AI_Brief_2023년12월호_F.pdf', 'page': 5, 'total_pages': 23, 'Author': 'dj', 'Creator': 'Hwp 2018 10.0.0.13462', 'Producer': 'Hancom PDF 1.3.0.542', 'CreationDate': "D:20231208132838+09'00'", 'ModDate': "D:20231208132838+09'00'", 'PDFVersion': '1.4'}
{'source': 'data/SPRI_AI_Brief_2023년12월호

In [6]:
for doc in docs:
    doc.metadata["filename"] = doc.metadata["source"]

<br>

### 데이터셋 생성

In [7]:
# !pip install -qU ragas

In [8]:
from ragas.testset import TestsetGenerator
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from ragas.testset.graph import KnowledgeGraph
from ragas.testset.persona import Persona

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_classic.text_splitter import RecursiveCharacterTextSplitter

c:\Users\user\anaconda3\envs\langai_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\user\anaconda3\envs\langai_env\Lib\site-packages\instructor\providers\gemini\client.py:5: FutureWarning: 

All support for the `google.generativeai` package has ended. It will no longer be receiving 
updates or bug fixes. Please switch to the `google.genai` package as soon as possible.
See README for more details:

https://github.com/google-gemini/deprecated-generative-ai-python/blob/main/README.md

  import google.generativeai as genai  # type: ignore[import-not-found]


#### Ragas 최신 버전의 내부 작동 프로세스
1. **문서 임베딩 및 관계 추출 (Graph Construction)**
- 입력된 모든 docs를 분석하여 핵심 개체(Entity)와 개념을 추출
- 문서 A와 문서 B가 연결되는 지점(예: 동일한 인물, 연속된 사건)을 찾아 선으로 연결 $\rightarrow$ Knowledge Graph

<br>

2. **페르소나 생성 (Persona Generation)**
- 이 데이터를 실제로 사용할 '가상의 사용자'들을 생성 (예: "이 문서를 처음 보는 초보자", "기술적인 질문을 던지는 전문가" 등)
- 이 페르소나들이 각자의 관점에서 질문을 던지게 하여 문제의 다양성을 확보

<br>

3. **진화적 질문 생성 (Evolution)**
- Simple: 그래프의 한 노드(정보)에서 바로 질문을 생성
- Reasoning: 노드 여러 개를 거쳐야 답이 나오는 논리적인 질문을 생성
- Multi-context: 서로 다른 문서 조각들에 흩어진 정보를 조합해야 풀 수 있는 고난도 문제를 생성

<br>

4. **검증 및 필터링 (Critic)**
- 만들어진 질문이 너무 쉽지는 않은지, 정답(`ground_truth`)이 문서 내에 확실히 존재하는지 비평기(Critic) LLM이 검수

<br>

1. 질문을 만들어낼 **'지능(LLM)'** 과 문서를 읽어낼 **'눈(Embedding)'** 을 결정
- `LangchainLLMWrapper` / `Wrapper`: Ragas는 전용 도구
 - Ragas가 이해할 수 있게 Langchain 객체에 **'어댑터(Wrapper)'**를 끼워주는 작업
- `TestsetGenerator`: `gpt-4o-mini`라는 지능과 `text-embedding-3-small`이라는 시력을 가진 **문제 출제 위원회(Generator)**를 구성

In [13]:
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o-mini"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings(model="text-embedding-3-small"))

generator = TestsetGenerator(
    llm=generator_llm,
    embedding_model=generator_embeddings,
    # language="korean"
)

C:\Users\user\AppData\Local\Temp\ipykernel_60740\2902844630.py:1: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use llm_factory instead: from openai import OpenAI; from ragas.llms import llm_factory; llm = llm_factory('gpt-4o-mini', client=OpenAI(api_key='...'))
  generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o-mini"))
C:\Users\user\AppData\Local\Temp\ipykernel_60740\2902844630.py:2: DeprecationWarning: LangchainEmbeddingsWrapper is deprecated and will be removed in a future version. Use the modern embedding providers instead: embedding_factory('openai', model='text-embedding-3-small', client=openai_client) or from ragas.embeddings import OpenAIEmbeddings, GoogleEmbeddings, HuggingFaceEmbeddings
  generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings(model="text-embedding-3-small"))


2. 테스트 셋 생성
- **눈(Embedding)** 으로 문서들 사이의 관계를 파악해 **지식 지도(Knowledge Graph)** 를 도출
- **지능(LLM)** 으로 "이 내용으로 어떻게 하면 어려운 문제를 낼까?" 고민하며 질문($Q$), 답변($A$), 그리고 근거 문장($Context$)을 쌍으로 생성

<br>

- 네트워크나 그래프 이론에서 Hop은 한 지점에서 다른 지점으로 이동할 때 거치는 '다리(Edge)'의 개수
- 1-Hop (Single-Hop): 질문에 대한 답이 하나의 노드(정보 덩어리) 안에 바로 있는 경우
  - 예) "A사의 24년 매출은?" $\rightarrow$ [A사 매출 노드]에서 즉시 확인 가능.
- 2-Hop 이상 (Multi-Hop): 여러 노드를 건너가며 정보를 조합해야 답이 나오는 경우
  - 예) "A사 매출에 가장 큰 기여를 한 제품의 개발팀장은 누구인가?"[A사 매출] 노드에서 [제품 명칭] 확인 (1-Hop)[제품 명칭] 노드에서 [담당 팀장] 확인 (2-Hop)

<br>

| 구분 | Hop 수 | 정보의 형태 | 난이도 | 주요 평가 지표 | 
| - | - | - | - | - |
| SingleHopSpecific| 1개 | 구체적 (Specific) | 하 | 정밀도 (Precision)|
| MultiHopSpecific | 2개 이상 | 구체적 (Specific)| 중 | 컨텍스트 재구성 (Context Relevancy)|
| MultiHopAbstract | 2개 이상 | 추상적 (Abstract) | 상 | 답변 충실도 (Answer Faithfulness)|

In [14]:
from ragas.testset import TestsetGenerator
from ragas.testset.synthesizers import (
    SingleHopSpecificQuerySynthesizer,
    MultiHopSpecificQuerySynthesizer,
    MultiHopAbstractQuerySynthesizer
)

In [15]:
query_distribution = [
    (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
    (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
    (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
]

In [ ]:
testset = generator.generate_with_langchain_docs(
    documents=docs,
    testset_size=10,
    query_distribution=query_distribution,
    language="korean"
)

Generating Samples: 100%|██████████| 11/11 [00:09<00:00,  1.20it/s]


In [18]:
test_df = testset.to_pandas()
test_df

,user_input,reference_contexts,reference,persona_name,query_style,query_length,synthesizer_name
0,What actions has President Biden taken regardi...,"[1. 정책/법제 2. 기업/산업 3. 기술/연구 4. 인력/교육\n 미국, 안전하...",President Biden has signed an executive order ...,AI Safety Advocate,WEB_SEARCH_LIKE,LONG,single_hop_specific_query_synthesizer
1,How does the recent AI executive order address...,[형평성과 시민권 향상 △소비자\n보호 △노동자 지원 △혁신과 경쟁 촉진 △국제협력...,The recent AI executive order includes measure...,Policy Analyst in Technology Ethics,WEB_SEARCH_LIKE,MEDIUM,single_hop_specific_query_synthesizer
2,What measures are included in the AI executive...,"[혁신과 경쟁 촉진 △국제협력을 골자로 함 £바이든 대통령, AI 행정명령 통해 안...",The AI executive order announced by President ...,Policy Analyst in Technology Ethics,PERFECT_GRAMMAR,SHORT,single_hop_specific_query_synthesizer
3,What is the focus of E.O. 14110 regarding AI t...,[분야의 전문 지식을 갖춘 외국인들이 미국에서 공부하고 취업할 수 있도록 지원 ☞ ...,"E.O. 14110 is focused on the safe, secure, and...",Policy Analyst in Technology Ethics,WEB_SEARCH_LIKE,MEDIUM,single_hop_specific_query_synthesizer
4,What role did Japan play in the G7's establish...,"[G7, 첨단 AI 시스템의 위험 관리를 위한 국제 행동강령 마련 n 주요 7개국(...",Japan was one of the major seven countries (G7...,Policy Analyst in Technology Ethics,WEB_SEARCH_LIKE,LONG,single_hop_specific_query_synthesizer
5,What are the implications of Samsung's on-devi...,[<1-hop>\n\nKEY Contents n 삼성전자가 온디바이스에서 작동 가능...,The introduction of Samsung's on-device AI mod...,NaN,NaN,NaN,multi_hop_specific_query_synthesizer
6,What significant developments in AI technology...,[<1-hop>\n\nKEY Contents n 삼성전자가 온디바이스에서 작동 가능...,"On November 8, 2023, during the 'Samsung AI Fo...",NaN,NaN,NaN,multi_hop_specific_query_synthesizer
7,What are the implications of the launch of the...,"[<1-hop>\n\n데이터 출처 탐색기, 광범위한 데이터셋 정보 제공을 통해 데이...",The launch of the Data Provenance Explorer pla...,NaN,NaN,NaN,multi_hop_specific_query_synthesizer
8,"영국 과학혁신기술부가 설립한 AI 안전 연구소의 목표는 무엇이며, AI 안전성 정상...",[<1-hop>\n\n1. 정책/법제 2. 기업/산업 3. 기술/연구 4. 인력/교...,영국 과학혁신기술부가 설립한 AI 안전 연구소의 목표는 첨단 AI의 위험을 이해하고...,NaN,NaN,NaN,multi_hop_abstract_query_synthesizer
9,What initiatives are being taken by the UK gov...,[<1-hop>\n\n1. 정책/법제 2. 기업/산업 3. 기술/연구 4. 인력/교...,The UK government is establishing the AI Safet...,NaN,NaN,NaN,multi_hop_abstract_query_synthesizer


In [19]:
test_df.head()

# DataFrame을 CSV 파일로 저장
test_df.to_csv("data/ragas_synthetic_dataset.csv", index=False)

<br>

<hr>

<br>

## 02. RAGAS를 이용한 평가

In [20]:
import pandas as pd
from datasets import Dataset
import ast

In [21]:
df = pd.read_csv("data/ragas_synthetic_dataset.csv")
df.head()

,user_input,reference_contexts,reference,persona_name,query_style,query_length,synthesizer_name
0,What actions has President Biden taken regardi...,"['1. 정책/법제 2. 기업/산업 3. 기술/연구 4. 인력/교육\n 미국, 안전...",President Biden has signed an executive order ...,AI Safety Advocate,WEB_SEARCH_LIKE,LONG,single_hop_specific_query_synthesizer
1,How does the recent AI executive order address...,['형평성과 시민권 향상 △소비자\n보호 △노동자 지원 △혁신과 경쟁 촉진 △국제협...,The recent AI executive order includes measure...,Policy Analyst in Technology Ethics,WEB_SEARCH_LIKE,MEDIUM,single_hop_specific_query_synthesizer
2,What measures are included in the AI executive...,"['혁신과 경쟁 촉진 △국제협력을 골자로 함 £바이든 대통령, AI 행정명령 통해 ...",The AI executive order announced by President ...,Policy Analyst in Technology Ethics,PERFECT_GRAMMAR,SHORT,single_hop_specific_query_synthesizer
3,What is the focus of E.O. 14110 regarding AI t...,['분야의 전문 지식을 갖춘 외국인들이 미국에서 공부하고 취업할 수 있도록 지원 ☞...,"E.O. 14110 is focused on the safe, secure, and...",Policy Analyst in Technology Ethics,WEB_SEARCH_LIKE,MEDIUM,single_hop_specific_query_synthesizer
4,What role did Japan play in the G7's establish...,"['G7, 첨단 AI 시스템의 위험 관리를 위한 국제 행동강령 마련 n 주요 7개국...",Japan was one of the major seven countries (G7...,Policy Analyst in Technology Ethics,WEB_SEARCH_LIKE,LONG,single_hop_specific_query_synthesizer


In [22]:
test_dataset = Dataset.from_pandas(df)
test_dataset

Dataset({
    features: ['user_input', 'reference_contexts', 'reference', 'persona_name', 'query_style', 'query_length', 'synthesizer_name'],
    num_rows: 11
})

In [23]:
def convert_to_list(example):
    contexts = ast.literal_eval(example["reference_contexts"])
    return {"reference_contexts": contexts}

In [24]:
test_dataset = test_dataset.map(convert_to_list)
print(test_dataset)

Map: 100%|██████████| 11/11 [00:00<00:00, 2153.74 examples/s]

Dataset({
    features: ['user_input', 'reference_contexts', 'reference', 'persona_name', 'query_style', 'query_length', 'synthesizer_name'],
    num_rows: 11
})


<br>

### 평가

In [25]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

<br>

1. **문서 로드**

In [26]:
loader = PyMuPDFLoader("data/SPRI_AI_Brief_2023년12월호_F.pdf")
docs = loader.load()

2. **문서 분할**

In [27]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)

3. **임베딩 생성**

In [28]:
embeddings = OpenAIEmbeddings()

4. **DB 생성 및 저장**
- 벡터스토어를 생성

In [29]:
vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)

5. **검색기 생성**
- 문서에 포함되어 있는 정보를 검색하고 생성

In [30]:
retriever = vectorstore.as_retriever()

6. **프롬프트 생성**

In [31]:
prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 

#Context: 
{context}

#Question:
{question}

#Answer:"""
)

7. **언어모델(LLM) 생성**

In [32]:
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

8. **체인 생성**

In [33]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

<br>

#### 배치 데이터셋 생성
- 다량의 질문을 한 번에 처리할 때 용이

In [34]:
batch_dataset = [question for question in test_dataset["user_input"]]
batch_dataset[:3]

['What actions has President Biden taken regarding the development and use of AI to ensure safety and reliability?',
 'How does the recent AI executive order address issues related to 주택?',
 'What measures are included in the AI executive order announced by President Biden?']

- `batch()` 를 호출하여 배치 데이터셋에 대한 답변

In [35]:
answer = chain.batch(batch_dataset)
answer[:3]

['President Biden has signed an executive order on October 30, 2023, to ensure the safe and reliable development and use of AI. The executive order outlines several actions, including:\n\n1. Establishing safety and security standards for AI systems.\n2. Enhancing privacy protection.\n3. Promoting fairness and civil rights.\n4. Protecting consumers.\n5. Supporting workers.\n6. Encouraging innovation and competition.\n7. Fostering international cooperation.\n\nAdditionally, the order requires companies developing powerful AI systems to share safety test results and key system information with the U.S. government. It also aims to establish standards and best practices for verifying the safety and reliability of AI systems and for marking AI-generated content.',
 'The recent AI executive order addresses issues related to housing by expanding measures to prevent discrimination and bias caused by the irresponsible use of AI in the housing sector. It aims to develop best practices for the use

- LLM 이 생성한 답변을 `'answer'` 컬럼에 저장

In [36]:
if "answer" in test_dataset.column_names:
    test_dataset = test_dataset.remove_columns(["answer"]).add_column("answer", answer)
else:
    test_dataset = test_dataset.add_column("answer", answer)

<br>

### 답변 평가

<br>

#### Context Recall
- **Context recall은 검색된 context가 LLM 이 생성한 답변과 얼마나 일치하는지를 측정**
  
  $\rightarrow$ **모범 답안(Ground Truth)의 내용 중 몇 %나 검색 결과(Context)에 들어있는가?**
- **이는 `user_input`, `reference_name` 및 검색된 `reference_contexts` 를 사용하여 계산되며, 값은 0에서 1 사이로, 높을수록 더 나은 성능을 의미**

$$
\text{context recall} = \frac{|\text{GT claims that can be attributed to context}|}{|\text{Number of claims in GT}|}
$$

$$
= \frac{|\text{그 팩트들 중 '검색된 문서'를 근거로 설명 가능한 것의 개수}|}{|\text{모범 답안에 들어있는 전체 팩트(주장)의 개수}|}
$$

- $\rightarrow$ **검색 시스템(Retriever)** 의 성능을 평가
- 예) 상황: 사용자가 "독도는 어느 나라 땅이야?"
  - 모범 답안 (GT): "독도는 대한민국의 영토이며, 행정구역상 경상북도 울릉군에 속합니다." (주장 2개)
  - 검색된 문서 (Context): "독도는 동해에 있는 섬으로 대한민국이 실효 지배 중입니다." (행정구역 정보 없음)
  
  $\rightarrow$ 이 경우, 모범 답안의 주장 2개 중 1개(대한민국 영토)만 검색 문서에 들어있으므로 **Context Recall은 0.5 (50%)**

<br>

#### Context Precision
- **Context Precision은 contexts 내의 ground-truth 관련 항목들이 상위 순위에 있는지를 평가하는 지표**
  
  $\rightarrow$ **정답에 결정적인 힌트가 들어있는 문서가 얼마나 앞페이지(상위 순위)에 배치되어 있는가?**

  (RAG 시스템에서 LLM은 앞쪽에 배치된 정보(Context)를 더 중요하게 생각하는 경향 (Lost in the Middle 현상))

- **`user_input`, `reference_name`, 그리고 `reference_contexts`를 사용하여 계산되며, 0에서 1 사이의 값**
- **높은 점수일수록 더 나은 정밀도를 의미**

<br>

- **관련 있는 문서($v_k=1$)가 나타날 때마다 그때의 정밀도($\text{Precision@k}$)를 기록해서 평균**
  - **$v_k \in {0, 1}$: 순위 $k$에 있는 문서가 정답과 관련이 있으면 1, 없으면 0**

$$
\text{Context Precision@K} = \frac{\sum_{k=1}^{K} (\text{Precision@k} \times v_k)}{\text{Total number of relevant items in the top K results}}
$$

$$
\text{Precision@k} = \frac{\text{true positives@k}}{(\text{true positives@k + false positives@k})} = \text{1등부터 k등까지 중에서 진짜 정답 관련 문서가 몇 \%인가}
$$

$\rightarrow$ **이 지표는 정보 검색 시스템에서 검색된 컨텍스트의 품질을 평가하는 데 사용**

$\rightarrow$ **관련 정보가 얼마나 정확하게 상위 순위에 배치되었는지를 측정함으로써 시스템의 성능을 판단**

<br>

#### Answer Relevancy
- **생성된 답변이 주어진  Prompt에 얼마나 적절한지 평가**
  
  $\rightarrow$ **답변 시스템의 성능을 평가하는 데 유용하며, 생성된 답변이 원래 질문의 의도를 얼마나 잘 반영하는지 측정**

- 목적 : 생성된 답변의 관련성
- 점수 해석 : 낮은 점수는 불완전하거나 중복 정보를 포함한 답변을, 높은 점수는 더 나은 관련성을 나타냄
- 계산에 사용되는 요소 : `user_input`, `refernce_contexts`, `answer`

<br>

- 원래 `user_input`과 `answer`를 기반으로 생성된 인공적인 질문들 간의 평균 코사인 유사도
  - $E_{g_i}$는 생성된 질문 $i$의 임베딩
  - $E_o$는 원래 질문의 임베딩
  - $N$은 생성된 질문의 수
- 실제로는 점수가 대부분 0과 1 사이에 있지만, 최대 -1에서 1 사이의 값을 가질 수 있음

$$
\text{answer relevancy} = \frac{1}{N} \sum_{i=1}^N \cos(E_{g_i}, E_o)
$$

$$
\text{answer relevancy} = \frac{1}{N} \sum_{i=1}^N \frac{E_{g_i} \cdot E_o}{|E_{g_i}| |E_o|}
$$

<br>

#### Faithfulness
- **생성된 답변의 사실적 일관성을, 주어진 컨텍스트와 비교하며 측정**
- 목적 : 답변의 사실적 일관성을 컨텍스트와 비교하며 평가
- 계산 요소 : 답변과 검색된 컨텍스트를 사용
- 점수 범위 : 0에서 1사이로 조정되며, 높을수록 더 좋음

<br>

- 계산
1. 생성된 답변에서 주장들을 식별
2. 각 주장들을 주어진 컨텍스트와 대조 검증하여 컨텍스트에 추론 가능한지 확인
3. 점수 계산

$$
\text{Faithfulness score} = \frac{|\text{Number of claims in the generated answer that can be inferred from given context}|}{|\text{Total number of claims in the generated answer}|}
$$

- 예시
1. 질문: "아인슈타인은 어디서, 언제 태어났나요?"
2. 컨텍스트: "알버트 아인슈타인(1879년 3월 14일 출생)은 독일 출신의 이론 물리학자로, 역사상 가장 위대하고 영향력 있는 과학자 중 한 명으로 여겨집니다."
3. 높은 충실도 답변: "아인슈타인은 1879년 3월 14일 독일에서 태어났습니다."
4. 낮은 충실도 답변: "아인슈타인은 1879년 3월 20일 독일에서 태어났습니다."

<br>

$\rightarrow$ **생성된 답변이 주어진 컨텍스트에 얼마나 충실한지를 평가하는 데 유용, 특히 질문-답변 시스템의 정확성과 신뢰성을 측정하는 데 중요**

<br>

```python
from ragas import evaluate
# from ragas.metrics.collections import answer_relevancy, faithfulness, context_recall, context_precision
from ragas.metrics import (
    ContextPrecision,
    Faithfulness,
    AnswerRelevancy,
    ContextRecall,
)

result = evaluate(
    dataset=test_dataset,
    metrics=[
        ContextPrecision(),
        Faithfulness(),
        AnswerRelevancy(),
        ContextRecall(),
    ],
    column_map={
        "retrieved_contexts": "reference_contexts", 
        "user_input": "user_input", 
        "reference": "reference",
    }
)

result_df = result.to_pandas()
result_df.head()

result_df.loc[:, "context_precision":"context_recall"]
```